# Translation Models


Machine translation is a pivotal field within natural language processing (NLP) that focuses on automating the conversion of text or speech from one language to another. It relies on sophisticated models and techniques to accomplish this challenging task effectively. One of the cornerstone methods in machine translation is the sequence-to-sequence (seq2seq) model, which employs deep neural networks to encode input text and then decode it into the target language. This technique has revolutionized translation tasks by learning to capture complex linguistic nuances and contextual information. Additionally, other models like Transformer-based models, including the famous BERT and GPT-3, have also made significant strides in translation, leveraging attention mechanisms to excel in various language pairs and domains. The choice of model depends on specific translation requirements, language pairs, and the quality of available training data. In this Colab file, we havee given a basic demo on how tto use the dataset and work on a simple seq2seq moel usig RNN.Your task will be to improve the model to the maximum you can ,make prediction on the test dataset given and write a code to generate the BLEU score of you prediction compared to original.






In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional,LSTM, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
##Loading and processing data
eng_fr = pd.read_csv("Dataset/nlp_intel_train.csv")
eng_fr_test = pd.read_csv("Dataset/nlp_intel_test.csv")

In [4]:
eng_fr

,Unnamed: 0,en,fr
0,1000,"In 1981, he founded the Astronomy Club of Rimo...","En 1981, il fonde le Club d'Astronomie de Rimo..."
1,1001,The club was very active and they twice organi...,Le club est très actif et organise à deux occa...
2,1002,"In 1983, Lemay initiated the first joint meeti...","En 1983, il est l'instigateur à Québec du cong..."
3,1003,"The conference took place in Quebec City, and ...",Le congrès est un franc succès et regroupe pas...
4,1004,"From 1990 to 1992, he was the National Preside...","De 1990 à 1992, il est président national de l..."
...,...,...,...
17995,18995,Imports of shrimp and prawn recorded also a sh...,"En 2001, une forte baisse des importations jap..."
17996,18996,The volume of import decreased by 16.3% from 9...,"En effet, entre 2000 et 2001, le volume des im..."
17997,18997,The market for northern shrimp (Pandalus borea...,"De plus, le marché mondial des crevettes nordi..."
17998,18998,Imports of molluscs (almost 100% of this being...,"Entre 2000 et 2001, les importations de mollus..."


In [5]:
eng_fr = eng_fr.dropna(axis=0, how="any", subset=None, inplace=False)
eng_fr_test = eng_fr_test.dropna(axis=0, how="any", subset=None, inplace=False)

In [6]:
##Tokenizer and padding

def tokenize(data):
  t = Tokenizer()
  t.fit_on_texts(data)
  return t


def training_sequences(tokenizer, m_length, data):
    seq = tokenizer.texts_to_sequences(data)
    seq = pad_sequences(seq, maxlen = m_length, padding='post')
    return seq


In [7]:
#Preprocessing by tokenization and padding
#return processed data and tokenizer
def preprocess(x, y):

    x_tk = tokenize(x)
    y_tk = tokenize(y)

    preprocess_x = training_sequences(x_tk,55,x)
    preprocess_y = training_sequences(y_tk,55,y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

In [8]:
preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer = preprocess(eng_fr["en"].tolist(), eng_fr["fr"].tolist())

In [9]:
preproc_english_sentences

array([[   4, 3430,  241, ...,    0,    0,    0],
       [   1, 2716,   28, ...,    0,    0,    0],
       [   4, 3431, 8489, ...,    0,    0,    0],
       ...,
       [   1,   19,    8, ...,    0,    0,    0],
       [  36,    2, 6919, ...,    0,    0,    0],
       [   2,    1,  332, ...,    0,    0,    0]])

In [10]:
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Max English sentence length: 55
Max French sentence length: 55
English vocabulary size: 21789
French vocabulary size: 27712


In [11]:
#Final output funtion
def logits_to_text(logits, tokenizer):

    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = ' '

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [12]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    learning_rate = 0.001

    # Build the layers
    model = Sequential()
    model.add(Embedding(french_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(GRU(256, return_sequences=True))
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(english_vocab_size, activation='softmax'))

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

In [13]:
preproc_french_sentences.shape[1]

55

In [14]:
tmp_x =pad_sequences(preproc_french_sentences, maxlen = 55, padding = 'post')
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train
model = bd_model(
    tmp_x.shape,
    preproc_english_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

model.summary()

model.fit(tmp_x, preproc_english_sentences, batch_size=64, epochs=5, validation_split=0.2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 55, 256)           7094528   
                                                                 
 gru (GRU)                   (None, 55, 256)           394752    
                                                                 
 dense (Dense)               (None, 55, 1024)          263168    
                                                                 
 dense_1 (Dense)             (None, 55, 21790)         22334750  
                                                                 
Total params: 30,087,198
Trainable params: 30,087,198
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
  2/225 [..............................] - ETA: 27:34 - loss: 9.9785 - accuracy: 0.3291      

KeyboardInterrupt: 

In [ ]:
i= 1


print("Prediction:")
print(logits_to_text(model.predict(tmp_x[[i]])[0], english_tokenizer))
print("\nCorrect Translation:")
print(eng_fr["en"].tolist()[i])
print("\nOriginal text:")
print(eng_fr["fr"].tolist()[i])

Prediction:
1/1 [==============================] - 0s 330ms/step
the club was is active and of the two the in of and 1997 the the of of the of of the amateur of of <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
The club was very active and they twice organized the annual conference of the Amateur Astronomy Federation of Quebec in 1990 and 1997.

Original text:
Le club est très actif et organise à deux occasions (en 1990 et 1997) le congrès annuel de la Fédération des Astronomes Amateurs du Québec.
